<a href="https://colab.research.google.com/github/nwferreri/segmentation-rfm/blob/main/segmentation_rfm_customers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentation - Recency, Frequency, Monetary (Customer Order Data)

Dataset with customer data.

1. Prepare basket varaible
2. Rename variables
3. Create RFM model with 3 levels
4. Define 3 segments
5. Prepare final table overview

##0. Directory, Libraries, Data

In [ ]:
# Directory
%cd /content/drive/MyDrive/ZTM/Python for Business/Segmentation/RFM

/content/drive/MyDrive/ZTM/Python for Business/Segmentation/RFM


In [ ]:
# Libraries
import pandas as pd

In [ ]:
# Data
df = pd.read_csv('customer_data.csv').dropna()
df.head()

,customer_id,revenue,most_recent_visit,number_of_orders,recency_days
0,22086,777,5/14/2006,9,232
1,2290,1555,9/8/2006,16,115
2,26377,336,11/19/2006,5,43
3,24650,1189,10/29/2006,12,64
4,12883,1229,12/9/2006,12,23


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39999 entries, 0 to 39998
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   customer_id        39999 non-null  int64 
 1   revenue            39999 non-null  int64 
 2   most_recent_visit  39999 non-null  object
 3   number_of_orders   39999 non-null  int64 
 4   recency_days       39999 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 1.5+ MB


##1. Prepare basket variable

In [ ]:
df['Monetary'] = df.revenue / df.number_of_orders
df.head()

,customer_id,revenue,most_recent_visit,number_of_orders,recency_days,Monetary
0,22086,777,5/14/2006,9,232,86.333333
1,2290,1555,9/8/2006,16,115,97.187500
2,26377,336,11/19/2006,5,43,67.200000
3,24650,1189,10/29/2006,12,64,99.083333
4,12883,1229,12/9/2006,12,23,102.416667


##2. Rename variables

In [ ]:
df.rename(columns={'recency_days': 'Recency',
                   'number_of_orders': 'Frequency'}, inplace=True)
df.head(1)

,customer_id,revenue,most_recent_visit,Frequency,Recency,Monetary
0,22086,777,5/14/2006,9,232,86.333333


In [ ]:
# Remove unwanted variables
df = df.drop(columns=['revenue', 'most_recent_visit'])
df.head(1)

,customer_id,Frequency,Recency,Monetary
0,22086,9,232,86.333333


##3. Create RFM model with 3 levels

In [ ]:
# Create groups
df['F'] = pd.qcut(x=df['Frequency'], q=3, labels=range(1, 4, 1))
df['R'] = pd.qcut(x=df['Recency'], q=3, labels=range(3, 0, -1))
df['M'] = pd.qcut(x=df['Monetary'], q=3, labels=range(1, 4, 1))
df.head(2)

,customer_id,Frequency,Recency,Monetary,F,R,M
0,22086,9,232,86.333333,2,1,1
1,2290,16,115,97.187500,3,2,2


In [ ]:
# Create RFM score
df['RFM'] = df[['R', 'F', 'M']].sum(axis=1)
df

,customer_id,Frequency,Recency,Monetary,F,R,M,RFM
0,22086,9,232,86.333333,2,1,1,4
1,2290,16,115,97.187500,3,2,2,7
2,26377,5,43,67.200000,1,3,1,5
3,24650,12,64,99.083333,3,3,2,8
4,12883,12,23,102.416667,3,3,2,8
...,...,...,...,...,...,...,...,...
39994,3249,10,31,99.800000,2,3,2,7
39995,6686,8,187,96.375000,1,2,2,5
39996,16418,9,154,112.888889,2,2,3,7
39997,9117,7,195,96.857143,1,2,2,5


##4. Define 3 segments

In [ ]:
# Create RFM function
def rfm_segment(df):
  '''
  Determines the rating of a given customer based on their RFM score.
  '''
  if df['RFM'] >= 8:
    return 'Superstar'
  elif ((df['RFM'] >= 5) and (df['RFM'] < 8)):
    return 'High Potential'
  else:
    return 'Low Relevance'

In [ ]:
# Apply RFM function
df['RFM_level'] = df.apply(rfm_segment, axis=1)
df

,customer_id,Frequency,Recency,Monetary,F,R,M,RFM,RFM_level
0,22086,9,232,86.333333,2,1,1,4,Low Relevance
1,2290,16,115,97.187500,3,2,2,7,High Potential
2,26377,5,43,67.200000,1,3,1,5,High Potential
3,24650,12,64,99.083333,3,3,2,8,Superstar
4,12883,12,23,102.416667,3,3,2,8,Superstar
...,...,...,...,...,...,...,...,...,...
39994,3249,10,31,99.800000,2,3,2,7,High Potential
39995,6686,8,187,96.375000,1,2,2,5,High Potential
39996,16418,9,154,112.888889,2,2,3,7,High Potential
39997,9117,7,195,96.857143,1,2,2,5,High Potential


##5. Prepare final table overview

In [ ]:
# Looking into the segments
df.groupby('RFM_level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
}).round(1)

Recency Frequency Monetary       
                  mean      mean     mean  count
RFM_level                                       
High Potential   171.8       9.8     97.0  26445
Low Relevance    306.6       7.1     78.5   7179
Superstar         80.1      12.8    108.3   6375